In [1]:
#! pip install 'h5py==2.10.0' --force-reinstall

In [2]:
%tensorflow_version 1.x

from google.colab import drive
drive.mount("/content/drive", force_remount=True)


import sys

PATH = '/content/drive/MyDrive/Colab Notebooks/CS598_DLH_Paper211'
sys.path.append(PATH)

TensorFlow 1.x selected.
Mounted at /content/drive


In [3]:
import pandas as pd
import os
import numpy as np
from gensim.models import Word2Vec, FastText
# import glove
# from glove import Corpus

import collections
import gc 

import keras
from keras import backend as K
from keras import regularizers
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Dropout, Input, concatenate, merge, Activation, Concatenate, LSTM, GRU
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Conv1D, BatchNormalization, GRU, Convolution1D, LSTM
from keras.layers import UpSampling1D, MaxPooling1D, GlobalMaxPooling1D, GlobalAveragePooling1D,MaxPool1D, merge

from keras.optimizers import Adam

from keras.callbacks import EarlyStopping, ModelCheckpoint, History, ReduceLROnPlateau
from keras.utils import np_utils
from keras.backend.tensorflow_backend import set_session, clear_session, get_session
import tensorflow as tf


from sklearn.utils import class_weight
from sklearn.metrics import average_precision_score, roc_auc_score, accuracy_score, f1_score

import warnings
warnings.filterwarnings('ignore')

SEED='multimodalavg'
SEED='245'

Using TensorFlow backend.


In [4]:
def create_dataset(dict_of_ner):
    temp_data = []
    for k, v in sorted(dict_of_ner.items()):
        temp = []
        for embed in v:
            temp.append(embed)
        temp_data.append(np.mean(temp, axis = 0)) 
    return np.asarray(temp_data)

def make_prediction_multi_avg(model, test_data):
    probs = model.predict(test_data)
    y_pred = [1 if i>=0.5 else 0 for i in probs]
    return probs, y_pred

def save_scores_multi_avg(predictions, probs, ground_truth, 
                          
                          embed_name, problem_type, iteration, hidden_unit_size,
                          
                          sequence_name, type_of_ner):
    
    auc = roc_auc_score(ground_truth, probs)
    auprc = average_precision_score(ground_truth, probs)
    acc   = accuracy_score(ground_truth, predictions)
    F1    = f1_score(ground_truth, predictions)
    
    result_dict = {}    
    result_dict['auc'] = auc
    result_dict['auprc'] = auprc
    result_dict['acc'] = acc
    result_dict['F1'] = F1
    
    result_path = PATH + f"/results/multimodalavg/{SEED}/"
    file_name = str(sequence_name)+"-"+str(hidden_unit_size)+"-"+embed_name
    file_name = file_name +"-"+problem_type+"-"+str(iteration)+"-"+type_of_ner+f"-avg-{SEED}.p"
    pd.to_pickle(result_dict, os.path.join(result_path, file_name))

    print(f'auc-{auc}, auprc-{auprc}, acc-{acc}, f1-{F1}')
    
def avg_ner_model(layer_name, number_of_unit, embedding_name):

    print(f'Inputs-{layer_name}, {number_of_unit}, {embedding_name}')
    if embedding_name == "concat":
        input_dimension = 200
    else:
        input_dimension = 100

    sequence_input = Input(shape=(24,104))

    input_avg = Input(shape=(input_dimension, ), name = "avg")        
#     x_1 = Dense(256, activation='relu')(input_avg)
#     x_1 = Dropout(0.3)(x_1)
    
    if layer_name == "GRU":
        x = GRU(number_of_unit)(sequence_input)
    elif layer_name == "LSTM":
        x = LSTM(number_of_unit)(sequence_input)

    x = keras.layers.Concatenate()([x, input_avg])

    x = Dense(256, activation='relu')(x)
    x = Dropout(0.2)(x)
    
    
    logits_regularizer = tf.contrib.layers.l2_regularizer(scale=0.01)
    
    preds = Dense(1, activation='sigmoid',use_bias=False, kernel_regularizer=logits_regularizer)(x)
    
    
    opt = Adam(lr=0.001, decay = 0.01)
    model = Model(inputs=[sequence_input, input_avg], outputs=preds)
    model.compile(loss='binary_crossentropy',
                  optimizer=opt,
                  metrics=['acc'])
    
    return model

In [5]:
# Reset Keras Session
def reset_keras(model):
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

    try:
        del model # this is from global space - change this as you need
    except:
        pass

    gc.collect() # if it's done something you should see a number being outputted

In [6]:
type_of_ner = "new"

x_train_lstm = pd.read_pickle(PATH + "/data/"+type_of_ner+"_x_train.pkl")
x_dev_lstm = pd.read_pickle(PATH + "/data/"+type_of_ner+"_x_dev.pkl")
x_test_lstm = pd.read_pickle(PATH + "/data/"+type_of_ner+"_x_test.pkl")

y_train = pd.read_pickle(PATH + "/data/"+type_of_ner+"_y_train.pkl")
y_dev = pd.read_pickle(PATH + "/data/"+type_of_ner+"_y_dev.pkl")
y_test = pd.read_pickle(PATH + "/data/"+type_of_ner+"_y_test.pkl")

ner_word2vec = pd.read_pickle(PATH + "/data/"+type_of_ner+"_ner_word2vec_limited_dict.pkl")
ner_fasttext = pd.read_pickle(PATH + "/data/"+type_of_ner+"_ner_fasttext_limited_dict.pkl")
ner_concat = pd.read_pickle(PATH + "/data/"+type_of_ner+"_ner_combined_limited_dict.pkl")

train_ids = pd.read_pickle(PATH + "/data/"+type_of_ner+"_train_ids.pkl")
dev_ids = pd.read_pickle(PATH + "/data/"+type_of_ner+"_dev_ids.pkl")
test_ids = pd.read_pickle(PATH + "/data/"+type_of_ner+"_test_ids.pkl")


In [7]:
embedding_types = ['word2vec', 'fasttext', 'concat']
embedding_dict = [ner_word2vec, ner_fasttext, ner_concat]
target_problems = ['mort_hosp', 'mort_icu', 'los_3', 'los_7']
target_problems = ['los_2', 'los_4','los_5']


num_epoch = 50
model_patience = 5
monitor_criteria = 'val_loss'
batch_size = 64
iter_num = 11
unit_sizes = [128, 256]

#layers = ["LSTM", "GRU"]
layers = ["GRU"]
for each_layer in layers:
    print ("Layer: ", each_layer)
    for each_unit_size in unit_sizes:
        print ("Hidden unit: ", each_unit_size)

        for embed_dict, embed_name in zip(embedding_dict, embedding_types):    
            print ("Embedding: ", embed_name)
            print("=============================")

            temp_train_ner = dict((k, embed_dict[k]) for k in train_ids)
            temp_dev_ner = dict((k, embed_dict[k]) for k in dev_ids)
            temp_test_ner = dict((k, embed_dict[k]) for k in test_ids)

            x_train_ner = create_dataset(temp_train_ner)
            x_dev_ner = create_dataset(temp_dev_ner)
            x_test_ner = create_dataset(temp_test_ner)

            for iteration in range(1, iter_num):
                print ("Iteration number: ", iteration)

                for each_problem in target_problems:
                    print ("Problem type: ", each_problem)
                    print ("__________________")

                    early_stopping_monitor = EarlyStopping(monitor=monitor_criteria, patience=model_patience)
                    best_model_name = "avg-"+str(embed_name)+"-"+str(each_problem)+"-"+"best_model.hdf5"
                    checkpoint = ModelCheckpoint(best_model_name, monitor='val_loss', verbose=1, save_best_only=True, mode='min', period=1)


                    callbacks = [early_stopping_monitor, checkpoint]

                    model = avg_ner_model(each_layer, each_unit_size, embed_name)
                    print(len(x_train_ner))
                    model.fit([x_train_lstm, x_train_ner], y_train[each_problem], epochs=num_epoch, verbose=1, 
                              validation_data=([x_dev_lstm, x_dev_ner], y_dev[each_problem]), callbacks=callbacks, batch_size=batch_size )

                    model.load_weights(best_model_name)

                    probs, predictions = make_prediction_multi_avg(model, [x_test_lstm, x_test_ner])
                    
                    save_scores_multi_avg(predictions, probs, y_test[each_problem], 
                               embed_name, each_problem, iteration, each_unit_size, 
                               each_layer, type_of_ner)
                    
                    reset_keras(model)
                    # del model
                    clear_session()
                    gc.collect()

Streaming output truncated to the last 5000 lines.

Epoch 00002: val_loss improved from 0.45066 to 0.44422, saving model to avg-concat-los_5-best_model.hdf5
Epoch 3/50
15219/15219 [==============================] - 6s 406us/step - loss: 0.4313 - acc: 0.8202 - val_loss: 0.4400 - val_acc: 0.8138

Epoch 00003: val_loss improved from 0.44422 to 0.43997, saving model to avg-concat-los_5-best_model.hdf5
Epoch 4/50
15219/15219 [==============================] - 6s 412us/step - loss: 0.4218 - acc: 0.8231 - val_loss: 0.4401 - val_acc: 0.8175

Epoch 00004: val_loss did not improve from 0.43997
Epoch 5/50
15219/15219 [==============================] - 6s 404us/step - loss: 0.4169 - acc: 0.8225 - val_loss: 0.4393 - val_acc: 0.8175

Epoch 00005: val_loss improved from 0.43997 to 0.43928, saving model to avg-concat-los_5-best_model.hdf5
Epoch 6/50
15219/15219 [==============================] - 6s 401us/step - loss: 0.4107 - acc: 0.8267 - val_loss: 0.4399 - val_acc: 0.8216

Epoch 00006: val_loss did 

In [8]:
df = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/CS598_DLH_Paper211/results/proposed/GRU-256-word2vec-los_3-1-new-avg-.p')
df

{'F1': 0.39043381535038935,
 'acc': 0.7479300827966882,
 'auc': 0.7174508392137696,
 'auprc': 0.5021809956612135}

In [17]:
PATH= '/content/drive/MyDrive/Colab Notebooks/CS598_DLH_Paper211/results/multimodalavg/245'

In [18]:
import glob
from pathlib import Path

results_dict = {'target_problem':[],'embeddings':[], 'F1':[],'acc':[],'auc':[],'auprc':[]}


files = glob.glob(f"{PATH}/*.p")
for file in files:
  file_name = Path(file).stem
  file_name_items = file_name.split('-')
  results_dict['target_problem'].append(file_name_items[3])
  results_dict['embeddings'].append(file_name_items[2])
  scores_dict = pd.read_pickle(file)
  results_dict['acc'].append(scores_dict['acc'])
  results_dict['auc'].append(scores_dict['auc'])
  results_dict['auprc'].append(scores_dict['auprc'])
  results_dict['F1'].append(scores_dict['F1'])

In [19]:
df=pd.DataFrame.from_dict(results_dict,orient='index').transpose()
df.shape

(180, 6)

In [20]:
df_groupby = df.groupby(['target_problem','embeddings']).agg({'auc':['mean','std'],'auprc':['mean','std'], 'F1':['mean','std']})
df_groupby_new = df.groupby(['target_problem','embeddings']).agg({'auc':['mean','std'],'auprc':['mean','std'], 'F1':['mean','std']}).reset_index()

In [21]:
new_columns = []
for i in df_groupby_new.columns:
  j=''
  if i[1]:
    j='_'.join(list(i))
  else:
    j=i[0]
  new_columns.append(j)

df_groupby_new.columns = new_columns

In [22]:
df_groupby_new


,target_problem,embeddings,auc_mean,auc_std,auprc_mean,auprc_std,F1_mean,F1_std
0,los_2,concat,0.684860,0.003845,0.838990,0.002265,0.819398,0.002847
1,los_2,fasttext,0.682578,0.005994,0.837696,0.003238,0.819406,0.004541
2,los_2,word2vec,0.686976,0.003881,0.840675,0.001974,0.820966,0.003018
3,los_4,concat,0.707979,0.002120,0.485638,0.006086,0.370976,0.014746
4,los_4,fasttext,0.708168,0.001935,0.490229,0.003054,0.376294,0.010587
5,los_4,word2vec,0.712958,0.002169,0.494491,0.004207,0.376814,0.016581
6,los_5,concat,0.713222,0.002694,0.372143,0.005134,0.224818,0.027128
7,los_5,fasttext,0.713122,0.002719,0.375503,0.004833,0.235139,0.020757
8,los_5,word2vec,0.719029,0.002911,0.378599,0.004456,0.236152,0.017882


In [23]:
df_groupby_new.groupby(['target_problem'])[['auc_mean','auprc_mean','F1_mean']].max()

,auc_mean,auprc_mean,F1_mean
target_problem,,,
los_2,0.686976,0.840675,0.820966
los_4,0.712958,0.494491,0.376814
los_5,0.719029,0.378599,0.236152
